### Package import

In [ ]:
# Caculation
import numpy as np
import re
from numpy import random

# Scraping
from bs4 import BeautifulSoup
import requests
import time
import urllib
from urllib.request import Request, urlopen, ProxyHandler, build_opener, install_opener, HTTPHandler
from urllib.error import HTTPError, URLError
from urllib.parse import quote

# Show Results
import pandas as pd
import matplotlib.pyplot as plt

### Set Parameters

In [ ]:
# Code Referenced & Edited from
# https://github.com/FelixMundial/SimpleDoubanScraper/tree/7e779d717a50a8fff68752ac90d00c26109a3468

#Setting User Agents

USER_AGENTS = [
    {'User-Agent': 'Mozilla/5.0 (Linux; Android 7.1.1; SM-A530F Build/NMF26X; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/64.0.3282.137 Mobile Safari/537.36 YandexSearch/7.21'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.12 Safari/535.11'},
    {'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:73.0) Gecko/20100101 Firefox/73.0'},
    {'User-Agent': 'Mozilla/5.0 (Linux; Android 10; P40) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/15.0 Chrome/90.0.4430.210 Mobile Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8) AppleWebKit/5331 (KHTML, like Gecko) Chrome/37.0.840.0 Mobile Safari/5331'},
    {'User-Agent': 'Mozilla/5.0 (Linux; Android 9; Redmi Note 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4603.0 Mobile Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)'},
    {'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12'},
    {'User-Agent': 'Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7'},
    {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}]

In [ ]:
#Scrape Free Proxy Site

url = "https://free-proxy-list.net/anonymous-proxy.html"
page = requests.get(url)
soup = BeautifulSoup(page.text,"html.parser")
proxy_df = soup.find("tbody")
proxy_single = proxy_df.find_all("tr")

#Set Proxies

proxy_list = []
for index in range(len(proxy_single)):
    if proxy_single[index].find_all("td")[6].text == "yes":
        address = str(proxy_single[index].find_all("td")[0].text)
        port = str(proxy_single[index].find_all("td")[1].text)
        #proxy_list.append(str("https://" + address + ":" + port))
        proxy_list.append(str(address + ":" + port))
PROXY_IPS = []
for index in range(len(proxy_list)):
    PROXY_IPS.append({"http": proxy_list[index]})

### Scraping Process

In [ ]:
#Request certain group discussion page

#First make the url for every page

group_discussion_url = 'http://www.douban.com/group/nopua/discussion?start='
group_disscusion_number = 0

In [ ]:
# Code Referenced & Edited from
# https://www.scrapehero.com/how-to-rotate-proxies-and-ip-addresses-using-python-3/

#Make Header & Proxie
headers = random.choice(USER_AGENTS)
proxies = random.choice(PROXY_IPS)
print("Connect to: " + str(proxies))
print("Header:" + str(headers))
print("------")

#Request & transfer the page (referenced from Github)
group_discussion_url = group_discussion_url + str(group_disscusion_number)
group_dis_req = requests.get(group_discussion_url, proxies=proxies, headers=headers)
group_dis_page_text = BeautifulSoup(group_dis_req.text,"html.parser")

In [ ]:
#Get certain group discussion post link into a dictionary

grp_dis_list = {}
list = group_dis_page_text.find_all("tr")
for i in list:
    if i["class"] == []:
        title = str(i.a["title"])
        link = i.a["href"]
        grp_dis_list[title]=link

In [ ]:
#Scrape Every Post

post_df = pd.DataFrame(columns = ["title","link","main text","reply text"])
for i in grp_dis_list:
    
    print("Scraping post: ", i )
    
    #Request & Get the html
    grp_dis_list[i] #get the url
    headers = random.choice(USER_AGENTS) #make a fake user
    proxies = random.choice(PROXY_IPS)
    req = requests.get(grp_dis_list[i], proxies=proxies, headers=headers)
    soup = BeautifulSoup(req.text,"html.parser")
    print("Connect to: " + str(proxies))
    print("Header:" + str(headers))
    print("------")
    
    # Post Title / Link
    post = [[]]
    post[0].append(i)
    post[0].append(grp_dis_list[i])
    
    # Post Main Text
    # get main post
    post_main = soup.find("div", class_="rich-content topic-richtext").find_all("p")
    # transfer main post object into a list of strings
    post_main_text = []
    for main in range(len(post_main)):
        post_main_text.append(post_main[main].text)
    
    # Post Reply Text
    #transfer reply post object into a list of strings
    post_reply = soup.find_all(class_="reply-content")
    post_reply_text = []
    for reply in range(len(post_reply)):
        post_reply_text.append(post_reply[reply].text)
    
    # Make the Dataset
    # add single post to data frame
    post[0].append(post_main_text)
    post[0].append(post_reply_text)
    post_df = post_df.append(pd.DataFrame(post, columns = ["title","link","main text","reply text"]))
    
    sleep_time = random.randint(10,20)
    print("Now sleeping for " + str(sleep_time) + "s...")
    time.sleep(sleep_time) # modified from Github Code

### Saving Results

In [ ]:
#Write to file
file_name = "DOUBAN_dataset_group03"
post_df.to_csv(file_name + ".tsv", sep = "\t", columns = ["title","link","main text","reply text"], index = False)